## Dependencies & Constantes

In [1]:
import os

import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors

import pandas as pd

import pickle

In [2]:
API_SERVICE = "youtube"
API_VERSION = "v3"
SCOPES = ["https://www.googleapis.com/auth/youtube.readonly"]

## Authentication

In [3]:
def authenticate():
  
  os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
  
  client_secrets_file = "client_secret.json"

  flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
      client_secrets_file, SCOPES)
  
  credentials = None
  
  token_file = "tocken.pickle"

  if os.path.exists(token_file):
    with open(token_file, 'rb') as token:
      credentials = pickle.load(token)

  if not credentials or not credentials.valid:
    credentials = flow.run_local_server()
    
    with open(token_file, "wb") as token:
      pickle.dump(credentials, token)
    
  return credentials 

CREDENTIALS = authenticate()

## Initialisation du client

In [4]:
CLIENT = googleapiclient.discovery.build(
      API_SERVICE, 
      API_VERSION, 
      credentials=CREDENTIALS
    )

## Noms de playlists & IDs

In [5]:
def get_playlist_ids():
  
  request = CLIENT.playlists().list(
      part="id, snippet",
      mine=True
  )
  response = request.execute()
  
  playlists = response.get("items")
  playlist_ids = [playlist["id"] for playlist in playlists]
  playlist_names = [playlist["snippet"]["title"] for playlist in playlists]
  
  return playlist_names, playlist_ids

## Titres des vidéos

In [6]:
def get_videos(playlist_id): 
  request = CLIENT.playlistItems().list(
    part="snippet",
    playlistId=playlist_id
  )
  response = request.execute()
  
  videos = response.get("items", [])
  
  return videos

## Arborescence

In [7]:
def set_tree(playlist_names, playlist_ids):
  playlist_dict = {}
  for i in range(len(playlist_names)):
    videos = get_videos(playlist_ids[i])
    
    video_titles = [video["snippet"]["title"] for video in videos]
    playlist_dict[playlist_ids[i]] = {
      "name": playlist_names[i],
      "videos": video_titles,
    }
  return playlist_dict

In [9]:
def main():
  playlist_names, playlist_ids = get_playlist_ids()
  
  playlist_dict = set_tree(playlist_names, playlist_ids)
  
  df = pd.DataFrame.from_dict(playlist_dict, orient="columns")
  
if __name__ == "__main__":
    main()

                       PLs-GpQgNJjGNDR-d6D31a4kzqWz_JutVb   
name                                             Hot Ones  \
videos  [HOT ONES : Mister V sera-t-il meilleur qu'au ...   

                       PLs-GpQgNJjGPIzZfX1hzEgKyN_DzSzmE4  
name                                                    a  
videos  [Full Stack Spotify Clone: Next 13.4, React, S...  
